In [1]:
import numpy as np
import pandas as pd
import json
import sklearn as sk
import matplotlib as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
def confusion_matrix(data, prompt):
    data = data[data["PromptType"]==prompt]
    return np.array([[len(data[data["model_bruteforce"]=="1"][data["human_bruteforce"]=="1"]), len(data[data["model_bruteforce"]=="1"][data["human_bruteforce"]=="0"])], [len(data[data["model_bruteforce"]=="0"][data["human_bruteforce"]=="1"]), len(data[data["model_bruteforce"]=="0"][data["human_bruteforce"]=="0"])]])

def correctness(data, prompt):
    data = data[data["PromptType"]==prompt]
    return len(data[data["correctness"]=="1"]) / len(data)

In [9]:
data = {}
summary = {}
models = ['DSChat', 'GeminiFlash', 'GeminiPro', 'o3', 'Qwen1', 'Qwen14', 'Qwen70']
# models = ['Qwen1']
for model in models:
    file_path = f"response_evaluation/Logic/SolutionSummary-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl"
    with open(file_path, 'r') as file:
        data[model] = [json.loads(line) for line in file]
    
    with open(f"response_evaluation/Logic/HintRedo-{model}/resultsEvaluations_evaluatedbyo3-2025-04-16.jsonl", 'r') as file:
        correctnessData = [json.loads(line) for line in file]
    
    prompts = pd.DataFrame(data[model])["PromptType"].unique()
    summaries = pd.DataFrame(data[model])
    correctness = pd.DataFrame(correctnessData)
    data[model] = correctness
    summary[model] = summaries
    print(model)
    
    scores = np.zeros((2, 3, 2))

    print("Adequate:", len(summaries[summaries["Summary"]=="1"]))
    print("Inadequate:", len(summaries[summaries["Summary"]=="0"]))
    for num in range(250):
        try:
            adequateSummary = int(summaries[summaries["ID"]==num]["Summary"].values[0])
        # print("Adequate Summary:", adequateSummary)
        # print(correctness[correctness["ID"]==num])

            # try:
            #     basic = int(correctness[correctness["PromptType"]=="basicprompt"][correctness["ID"]==num]["correctness"].values[0])
            #     scores[adequateSummary][0][:basic+1] += 1
            # except:
            #     pass
            # try:
            #     math = int(correctness[correctness["PromptType"]=="mathPrompt"][correctness["ID"]==num]["correctness"].values[0])
            #     scores[adequateSummary][1][:math+1] += 1
            # except:
            #     pass
            try:
                hint = int(correctness[correctness["PromptType"]=="hintPrompt"][correctness["ID"]==num]["correctness"].values[0])
                scores[adequateSummary][2][:hint+1] += 1
            except:
                pass
        except:
            pass
        
        # scores[adequateSummary][0][basic] += 1
        # scores[adequateSummary][1][math] += 1
        # scores[adequateSummary][2][hint] += 1
    
    
    print(scores)

DSChat
Adequate: 183
Inadequate: 67
[[[  0.   0.]
  [  0.   0.]
  [ 67.  18.]]

 [[  0.   0.]
  [  0.   0.]
  [182.  85.]]]
GeminiFlash
Adequate: 158
Inadequate: 65
[[[  0.   0.]
  [  0.   0.]
  [ 65.   4.]]

 [[  0.   0.]
  [  0.   0.]
  [158.  27.]]]
GeminiPro
Adequate: 20
Inadequate: 230
[[[  0.   0.]
  [  0.   0.]
  [230.  36.]]

 [[  0.   0.]
  [  0.   0.]
  [ 20.   3.]]]
o3
Adequate: 190
Inadequate: 55
[[[  0.   0.]
  [  0.   0.]
  [ 55.  22.]]

 [[  0.   0.]
  [  0.   0.]
  [190. 126.]]]
Qwen1
Adequate: 11
Inadequate: 239
[[[  0.   0.]
  [  0.   0.]
  [239.  10.]]

 [[  0.   0.]
  [  0.   0.]
  [ 11.   4.]]]
Qwen14
Adequate: 100
Inadequate: 150
[[[  0.   0.]
  [  0.   0.]
  [150.  25.]]

 [[  0.   0.]
  [  0.   0.]
  [100.  46.]]]
Qwen70
Adequate: 111
Inadequate: 139
[[[  0.   0.]
  [  0.   0.]
  [139.  23.]]

 [[  0.   0.]
  [  0.   0.]
  [111.  45.]]]


In [ ]:
fulldata = pd.DataFrame()
for dataset in data.keys():
    fulldata = pd.DataFrame(data[dataset])
    prompts = fulldata["PromptType"].unique()
    
    # fig, axes = plt.pyplot.subplots(1, len(prompts), figsize=(30, 6))
    for i in range(len(prompts)):
        prompt = prompts[i]
        cmd = sk.metrics.ConfusionMatrixDisplay(confusion_matrix(fulldata, prompt), display_labels=["Used Brute Force", "Did Not Use Brute Force"])
        cmd.plot()
        cmd.ax_.set(xlabel='Human Solution', ylabel='Model Solution')
        cmd.ax_.set_title(f'{dataset} Brute Force Matrix for {prompt}')
    
        print(f"Correctness for {dataset} on {prompt}:", correctness(fulldata, prompt))

In [ ]:
fig, axes = plt.pyplot.subplots(1, 5, figsize=(30, 6))
for i in range(len(prompts)):
    prompt = prompts[i]
    cmd = sk.metrics.ConfusionMatrixDisplay(confusion_matrix(fulldata, prompt), display_labels=["Used Brute Force", "Did Not Use Brute Force"])
    cmd.plot()
    cmd.ax_.set(xlabel='Human Solution', ylabel='Model Solution')
    cmd.ax_.set_title(f'Brute Force Matrix for {prompt}')

In [ ]:
tempdata = fulldata[fulldata["PromptType"] == "basicprompt"]
# print(tempdata.head())
len(tempdata[tempdata["model_bruteforce"]=="1"][tempdata["human_bruteforce"]=="0"])